# Video Explanation

I creted this

[https://www.youtube.com/watch?v=7VSuYIizrdA](https://www.youtube.com/watch?v=7VSuYIizrdA)

In [ ]:
import kagglehub

path = kagglehub.dataset_download("dimensi0n/imagenet-256")
print("Path to dataset files:", path)

100%|██████████| 7.15G/7.15G [01:00<00:00, 126MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dimensi0n/imagenet-256/versions/1


In [ ]:
from keras.utils import image_dataset_from_directory

"""
split train, and test dataset, there are tensors of 32 images batch
each element of this tensor have element[0] = tensor of 32 images with (256,256,3) shape
and element[1] = tensor of 32 interger values these are classes those respective image belong
"""
(train_dataset_org, test_dataset_org) = image_dataset_from_directory(
    directory = path,
    validation_split = 0.2,
    subset= 'both', # return both train, and test
    labels = 'inferred',
    label_mode = 'categorical',
    batch_size = 500, # how many images to be considered per element of this dataset
    image_size = (256, 256),
    seed = 123
)

Found 539826 files belonging to 1000 classes.
Using 431861 files for training.
Using 107965 files for validation.


In [ ]:
from tensorflow import cast, float32

# you know these images pixel range from 0 to 255, so you need to scale it down b/w 0 and 1.
train_dataset = train_dataset_org.map(lambda x, y: (cast(x/255.0, float32), y))
test_dataset = test_dataset_org.map(lambda x, y: (cast(x/255.0, float32), y))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x

    # First 3x3 convolution
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Second 3x3 convolution
    x = layers.Conv2D(filters, kernel_size, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # If shape/number of channels changed, adjust the shortcut:
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same', use_bias=False)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    # Elementwise add + final ReLU
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

def inception_block(x, filters):
    # Branch 1: 1x1
    branch1 = layers.Conv2D(filters // 4, 1, padding='same', activation='relu')(x)

    # Branch 2: 1x1
    branch2 = layers.Conv2D(filters // 4, 1, padding='same', activation='relu')(x)
    branch2 = layers.Conv2D(filters // 4, 3, padding='same', activation='relu')(branch2)

    # Branch 3: 1x1
    branch3 = layers.Conv2D(filters // 4, 1, padding='same', activation='relu')(x)
    branch3 = layers.Conv2D(filters // 4, 5, padding='same', activation='relu')(branch3)

    # Branch 4: 3x3
    branch4 = layers.MaxPooling2D(3, strides=1, padding='same')(x)
    branch4 = layers.Conv2D(filters // 4, 1, padding='same', activation='relu')(branch4)

    # Concatenate all branches
    return layers.Concatenate()([branch1, branch2, branch3, branch4])

def build_cnn(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Initial Conv + Pooling
    x = layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # 2 Residual Blocks
    x = residual_block(x, 64, stride=1)
    x = residual_block(x, 128, stride=2)  # downsample here

    # One Inception‐style Block
    x = inception_block(x, 256)

    # Transition layer to reduce channels/dimension
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, 1, padding='same', use_bias=False)(x)
    x = layers.AveragePooling2D(2, strides=2, padding='same')(x)

    # 2 More Residual Blocks
    x = residual_block(x, 256, stride=1)
    x = residual_block(x, 512, stride=2)  # further downsampling

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return Model(inputs, outputs)

In [ ]:
model = build_cnn(input_shape=(256, 256, 3), num_classes=1000)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, validation_data=test_dataset, epochs=3)

Epoch 1/3
864/864 ━━━━━━━━━━━━━━━━━━━━ 1270s 1s/step - accuracy: 0.0635 - loss: 5.4559 - val_accuracy: 0.1270 - val_loss: 4.6936
Epoch 2/3
864/864 ━━━━━━━━━━━━━━━━━━━━ 1187s 1s/step - accuracy: 0.2386 - loss: 3.6992 - val_accuracy: 0.2574 - val_loss: 3.5937
Epoch 3/3
864/864 ━━━━━━━━━━━━━━━━━━━━ 1230s 1s/step - accuracy: 0.3557 - loss: 2.9456 - val_accuracy: 0.2683 - val_loss: 3.6202


In [ ]:
model.save("1000-classes-model.h5")

In [ ]:
model.fit(train_dataset, validation_data=test_dataset, epochs=5, initial_epoch=3)

Epoch 4/5
864/864 ━━━━━━━━━━━━━━━━━━━━ 1189s 1s/step - accuracy: 0.4358 - loss: 2.4915 - val_accuracy: 0.3129 - val_loss: 3.3009
Epoch 5/5
506/864 ━━━━━━━━━━━━━━━━━━━━ 6:47 1s/step - accuracy: 0.4887 - loss: 2.2011